In [1]:
import numpy as np
import pandas as pd
from math import ceil
import psycopg2
from time import time
import psycopg2.extras
from copy import copy
import matplotlib.pyplot as plt
import seaborn as sns

sqluser = 'postgres'
dbname = 'test'
schema_name = 'mimiciii'
password = 'postgres'
db_schema = 'set search_path to ' + schema_name + ';'
con = psycopg2.connect(dbname=dbname, user=sqluser, password=password, host='/var/run/postgresql/')
cursor = con.cursor()
con.commit()




/home/ubuntu/anaconda3/envs/eric/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/eric/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [38]:
string = '''
select * from inputevents_mv limit 1000;
'''

df = pd.read_sql(string,con)

In [39]:
df.rateuom.unique()

array([None, 'mL/hour'], dtype=object)

In [53]:
ddf = pd.read_csv('/home/ubuntu/munge/nitem_mapping.csv')

In [54]:
dddf = ddf[ddf.table == 'outputevents']

In [62]:
len(dddf.itemid.unique().tolist())

61

In [68]:
string = '''
select * from d_items;
'''

ef = pd.read_sql(string,con)

In [69]:
eef = ef[ef.linksto == "outputevents"]

In [64]:
len(ef.itemid.unique().tolist())

48

In [71]:
eeff = eef[eef.dbsource == 'metavision']
eeff

,row_id,itemid,label,abbreviation,dbsource,linksto,category,unitname,param_type,conceptid
8727,14560,226607,Cerebral Ventricular #2,Cerebral Ventricular #2,metavision,outputevents,Drains,mL,Numeric,None
8728,14561,226608,Cerebral Subdural #1,Cerebral Subdural #1,metavision,outputevents,Drains,mL,Numeric,None
8729,14562,226609,Cerebral Subdural #2,Cerebral Subdural #2,metavision,outputevents,Drains,mL,Numeric,None
8730,14563,226610,Lumbar,Lumbar,metavision,outputevents,Drains,mL,Numeric,None
8731,14564,226612,Pericardial,Pericardial,metavision,outputevents,Drains,mL,Numeric,None
8732,14565,226613,Wound Vac #1,Wound Vac #1,metavision,outputevents,Drains,mL,Numeric,None
8733,14566,226614,Wound Vac #2,Wound Vac #2,metavision,outputevents,Drains,mL,Numeric,None
8734,14567,226616,Anderson,Anderson,metavision,outputevents,Drains,mL,Numeric,None
8735,14568,226617,Sump #1,Sump #1,metavision,outputevents,Drains,mL,Numeric,None
8736,14569,226618,Sump #2,Sump #2,metavision,outputevents,Drains,mL,Numeric,None


In [2]:
df = pd.read_csv('/home/ubuntu/analyze_shallow/output/contextwithicd9_fixed.csv')

In [4]:
df.columns.tolist()

['index',
 'icustay_id',
 'hadm_id',
 'subject_id',
 'first_wardid',
 'last_wardid',
 'hospital_expire_flag',
 'has_chartevents_data',
 'expire_flag',
 'ser_ind',
 'age_at_death',
 'age_at_adm',
 'los_total',
 'died',
 'adm_rank',
 'icu_rank',
 'icd9_code_0041_x',
 'icd9_code_00845',
 'icd9_code_00863',
 'icd9_code_0088',
 'icd9_code_0329',
 'icd9_code_0340',
 'icd9_code_035',
 'icd9_code_0380',
 'icd9_code_03811',
 'icd9_code_03812',
 'icd9_code_03819',
 'icd9_code_0382',
 'icd9_code_03840',
 'icd9_code_03842',
 'icd9_code_03843',
 'icd9_code_03844',
 'icd9_code_03849',
 'icd9_code_0388',
 'icd9_code_0389',
 'icd9_code_04082',
 'icd9_code_04100',
 'icd9_code_04101',
 'icd9_code_04102',
 'icd9_code_04104',
 'icd9_code_04109',
 'icd9_code_04110',
 'icd9_code_04111',
 'icd9_code_04112',
 'icd9_code_04119',
 'icd9_code_0412',
 'icd9_code_0413',
 'icd9_code_0414',
 'icd9_code_04149',
 'icd9_code_0415',
 'icd9_code_0416',
 'icd9_code_0417',
 'icd9_code_04182',
 'icd9_code_04183',
 'icd9_cod